In [ ]:
import gradio as gr
import torch
from transformers import BertForSequenceClassification, BertTokenizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [ ]:
dataset_name = 'Amazon_Unlocked_Mobile.csv'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

device = torch.device('cpu')
# 加載模型架構和權重
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# 加載訓練好的權重
saved_model_path = 'models/Amazon_Unlocked_Mobile.csv_bert.pth'
model.load_state_dict(torch.load(saved_model_path, map_location=device))

# 切換模型到評估模式
model.eval()
model.to(device)

print("Model loaded successfully and ready for inference.")


In [ ]:
def preprocess_text_batch(texts, tokenizer, max_length=128):
    encodings = tokenizer.batch_encode_plus(
        texts,
        add_special_tokens=True,
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    return encodings['input_ids'], encodings['attention_mask']

In [ ]:
def predict_sentiment(model, examples, tokenizer, batch_size=16):
    model.to(device)
    model.eval()
    results = []

    # 將文本分批處理
    for i in range(0, len(examples), batch_size):
        batch_examples = examples[i:i + batch_size]

        # 預處理文本批次
        input_ids, attention_mask = preprocess_text_batch(batch_examples, tokenizer)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        with torch.no_grad():
            # 模型預測
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # 使用 softmax 獲取每個類別的概率
            probabilities = torch.softmax(logits, dim=1)
            predicted_classes = probabilities.argmax(dim=1)  # 獲取預測的類別

            # 將結果轉換為列表，並加入結果
            results.extend(predicted_classes.cpu().numpy())

    return results

In [ ]:
# 定義評論分析函數
def analyze_sentiment(review):
    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # 模型推理
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    logits = outputs.logits

    probabilities = torch.softmax(logits, dim=1)
    predicted_class = probabilities.argmax(dim=1).item()

    if predicted_class == 0:
        return "負面"
    elif predicted_class == 1:
        return "中立"
    else:
        return "正面"

In [ ]:
# 讀取CSV檔案
bert_data = pd.read_csv('bert_reputation.csv')
chat_data = pd.read_csv('chatgpt_reputation.csv')

In [ ]:
# 初始化存放每年資料的字典
monthly_data = {}

# 定義年份範圍
years = list(bert_data.columns)

In [ ]:
# 儲存每一年資料到 monthly_data 字典
for year in years:
    # 提取 BERT 和 ChatGPT 的月份和評分資料
    bert_months = list(map(int, eval(bert_data.loc[0, year])))  # 提取 BERT 模型的月份並轉為整數
    bert_scores = eval(bert_data.loc[1, year])  # 提取 BERT 模型的評分
    chatgpt_months = list(map(int, eval(chat_data.loc[0, year])))  # 提取 ChatGPT 模型的月份並轉為整數
    chatgpt_scores = eval(chat_data.loc[1, year])  # 提取 ChatGPT 模型的評分

    # 排序 ChatGPT 的月份和評分
    month_score_pairs = sorted(zip(chatgpt_months, chatgpt_scores), key=lambda x: x[0])

    # 將排序後的月份和評分分開儲存
    chatgpt_months, chatgpt_scores = zip(*month_score_pairs)

    # 確保兩個模型的月份一致
    if bert_months == list(chatgpt_months):
        print(True)
        bert_scores = np.array(bert_scores)  # 將 bert_scores 轉換為 numpy array
        chatgpt_scores = np.array(chatgpt_scores)  # 將 chatgpt_scores 轉換為 numpy array

        # 逐元素相加並取平均值
        avg_scores = (bert_scores + chatgpt_scores) / 2
        avg_scores = avg_scores.tolist()

    else:
        print(f"月份不匹配: {year}")
        continue  # 當月份不匹配時跳過該年份

    # 提取其他評分數據
    total_reviews = bert_data.loc[2, year]  # 提取總評論數
    positive_reviews = bert_data.loc[3, year]  # 提取正面評論數
    negative_reviews = bert_data.loc[4, year]  # 提取負面評論數
    neutral_reviews = bert_data.loc[5, year]  # 提取中立評論數




    # 將結果存入 monthly_data 字典
    monthly_data[year] = {
        'months': bert_months,
        'scores': bert_scores,
        'total_reviews': total_reviews,
        'positive_reviews': positive_reviews,
        'negative_reviews': negative_reviews,
        'neutral_reviews': neutral_reviews
    }

In [ ]:
# 讀取 iPhone 發表會的日期 CSV
iphone_dates = pd.read_csv('iphone.csv')
iphone_dates.columns = iphone_dates.columns.str.strip()

In [ ]:
def plot_yearly_scores(selected_year):
    data = monthly_data[selected_year]
    months = data['months']
    scores = data['scores']
    total_reviews = data['total_reviews']
    positive_reviews = data['positive_reviews']
    negative_reviews = data['negative_reviews']
    neutral_reviews = data['neutral_reviews']

# 檢查 selected_year 資料類型
    print(f"Selected Year: {selected_year}, Type: {type(selected_year)}")

    plt.figure(figsize=(10, 6))
    plt.plot(months, scores, marker='o', color='blue')  # 繪製折線圖

    # 標註 iPhone 發表會日期
    for index, row in iphone_dates.iterrows():
        event_year = str(row['Year'])
        if event_year == selected_year:
            print("Entered the if block")
            event_date = pd.to_datetime(row['EventDate'])
            event_month = event_date.month
            iphone_model = row['iPhone']  # 取得 iPhone 型號

            print(f"Year: {event_year}, Month: {event_month}, iPhone: {row['iPhone']}")

            # 使用顏色填充發表會的月份區域
            plt.axvspan(event_month - 0.3, event_month + 0.3, color='lightblue', alpha=0.3, label=f'iPhone {iphone_model}')





    plt.title(f'Brand Reputation Scores for Apple in {selected_year}')
    plt.xlabel('Month')
    plt.ylabel('Score')
    plt.grid(True)

    plt_path = os.path.abspath(f"images/{selected_year}plot.png")
    plt.savefig(plt_path)
    plt.close()

    summary = (f"Total Reviews: {total_reviews}\n"
            f"Positive Reviews: {positive_reviews}\n"
            f"Negative Reviews: {negative_reviews}\n"
            f"Neutral Reviews: {neutral_reviews}")


    return plt_path, summary

In [ ]:
brands = ["Apple"]
def update_year_dropdown(brand):
    if brand == "Apple":  # 當選擇品牌是「Apple」時，顯示年份選單
        return gr.update(visible=True), gr.update(visible=True)
    else:  # 如果沒有選「Apple」，不顯示年份選單
        return gr.update(visible=False), gr.update(visible=False)

# 整合 Gradio 介面
def gradio_interface():
    with gr.Blocks(css="custom.css") as demo:

        # 上部: 輸入評論與情感分析
        with gr.Row():
            with gr.Column(scale=3):
                review_input = gr.Textbox(label="輸入評論", lines=3, placeholder="輸入評論後按下情感分析")
            with gr.Column(scale=1):
                result_output = gr.Textbox(label="情感分析結果", interactive=False, placeholder="結果會顯示在這裡")
                review_button = gr.Button("進行情感分析")
                review_button.click(fn=analyze_sentiment, inputs=review_input, outputs=result_output)

        # 中部: 品牌與年份選擇
        with gr.Row():
            brand_dropdown = gr.Dropdown(choices=brands, label="Select Brand")
            year_dropdown = gr.Dropdown(choices=years, label="Select Year")

        # 下部: 折線圖與評論數據
        with gr.Row():
            plot_output = gr.Image(visible=True, width=600, height=400)
            review_summary = gr.Textbox(label="評論總結", interactive=False, visible=True, lines=4)

        # 當選擇品牌時，更新年份選單的顯示
        brand_dropdown.change(fn=update_year_dropdown, inputs=brand_dropdown, outputs=[year_dropdown, plot_output])

        # 更新圖表和評論總結
        year_dropdown.change(fn=lambda year: plot_yearly_scores(year), inputs=year_dropdown, outputs=[plot_output, review_summary])

    return demo



In [ ]:
# 啟動 Gradio 介面
demo = gradio_interface()
demo.launch(share=True)